In [5]:
import pandas as pd
import yfinance as yf
from sklearn.metrics import accuracy_score
from sklearn.ensemble import ExtraTreesClassifier
from datetime import datetime
from sklearn.model_selection import train_test_split
from ml.features.preprocessing import calc_target, calc_indicators

import logging
import autosklearn.classification
import sklearn.metrics

from tsfresh import extract_features


In [6]:
#!pip install --upgrade setuptools wheel
#!pip install auto-sklearn

In [7]:
stock_data = yf.download("^GSPC", "1980-01-01", end=datetime.now().strftime('%Y-%m-%d'))
extra_features = stock_data.copy()

stock_data = calc_target(df=stock_data)
stock_data = calc_indicators(df=stock_data)

[*********************100%%**********************]  1 of 1 completed


/home/erik/DataspellProjects/stock-prediction/ml/features/preprocessing.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f"SMA {time_period}"] = ta.SMA(inputs, timeperiod=time_period)
/home/erik/DataspellProjects/stock-prediction/ml/features/preprocessing.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f"EMA {time_period}"] = ta.EMA(inputs, timeperiod=time_period)
/home/erik/DataspellProjects/stock-prediction/ml/features/preprocessing.py:54: SettingWithCopyWarning: 
A value is trying to be se

In [8]:
extra_features["id"] = extra_features.index
extra_features = extra_features.drop(["Open", "Adj Close"], axis=1)

KeyError: "['Open', 'Adj Close'] not found in axis"

In [10]:
extra_features

,High,Low,Close,Volume,id
Date,,,,,
1980-01-02,108.430000,105.290001,105.760002,40610000,1980-01-02
1980-01-03,106.080002,103.260002,105.220001,50480000,1980-01-03
1980-01-04,107.080002,105.089996,106.519997,39130000,1980-01-04
1980-01-07,107.800003,105.800003,106.809998,44500000,1980-01-07
1980-01-08,109.290001,106.290001,108.949997,53390000,1980-01-08
...,...,...,...,...,...
2023-11-27,4560.520020,4546.319824,4550.430176,3403990000,2023-11-27
2023-11-28,4568.140137,4540.509766,4554.890137,3586240000,2023-11-28
2023-11-29,4587.640137,4547.149902,4550.580078,4418760000,2023-11-29


In [ ]:
X = extract_features(extra_features, column_id="id")

Feature Extraction: 100%|██████████| 15/15 [05:01<00:00, 20.12s/it]


In [1]:
extra_features

NameError: name 'extra_features' is not defined

In [3]:
X

NameError: name 'X' is not defined

In [ ]:
stock_data = stock_data.join(X)

In [6]:
sp500_data = pd.read_csv('../../data/sp500_training_data_to_2023-11-15.csv')
sp500_data.set_index("Date", drop=True, inplace=True)

In [ ]:
X = sp500_data.drop(['Target'], axis=1)
train_x, test_x, train_y, test_y = train_test_split(X, sp500_data['Target'], test_size=0.25, random_state=42)

In [7]:
etc = ExtraTreesClassifier(n_estimators=500, random_state=42)
etc.fit(train_x, train_y)
y_pred = etc.predict(test_x)
accuracy = accuracy_score(test_y, y_pred)
print(f"Genauigkeit: {accuracy}")

Genauigkeit: 0.8249818445896877


## AutoML

In [8]:
# logging.basicConfig(level=logging.INFO)
# automl = autosklearn.classification.AutoSklearnClassifier()
# automl.fit(train_x, train_y)
# y_hat = automl.predict(test_x)
# print("Accuracy score", sklearn.metrics.accuracy_score(test_y, y_hat))

In [9]:
#print(automl.leaderboard())